# Managing custom Jupyter environments with Singularity
---
In this tutorial we will cover the basic workflow for managing custom software environments for Jupyter Notebooks using Singularity.

Singularity is a developing platform, so version matters **a lot**. The version we will be using is the latest development HEAD of 2.3.

In [3]:
singularity --version

2.3-HEAD.gadf5259


## Pull the base image from ~~SingularityHub~~ DockerHub
We maintain a base image on SingularityHub for running Jupyter* (https://singularity-hub.org/collections/440/). This image contains the minimum dependencies and configuration needed to run containerized Notebooks (standalone or JupyterHub-spawned), and is intended to serve as a base for user-built software environments.

At this time though bootstrapping from SingularityHub is still an upcoming feature (https://github.com/singularityware/singularity/issues/833) so we will instead be using the `jupyter/base-notebook` docker conatainer as a base image. The base-notebook is provided by the Jupyter Docker Stacks project (https://github.com/jupyter/docker-stacks), which provides pre-built stacks ready to be run standalone or behind JupyterHub.

### A basic pull
Start by pulling the Jupyter base image from SingularityHub. We specifically want commit _ae885c0a6226_, so we'll specify that by adding the `:ae885c0a6226` tag to the end of the repo path:

In [1]:
singularity pull --name "jupyter-base.img" docker://jupyter/base-notebook:ae885c0a6226

Initializing Singularity image subsystem
Opening image file: jupyter-base.img
Creating 892MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-base.img
Docker image path: index.docker.io/jupyter/base-notebook:ae885c0a6226
Cache folder set to /home/vagrant/.singularity/docker
Importing: base Singularity environment
Importing: /home/vagrant/.singularity/docker/sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Importing: /home/vagrant/.si

There it is! Your container is good to go.

In [2]:
singularity exec -e jupyter-base.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec lab labextension labhub
migrate nbconvert nbextension notebook run serverextension troubleshoot trust


## Customizing the base image
The base image is meant to capture the _minimum_ config and dependencies to run Jupyter Notebooks. Here we detail how to customize the base image to better suit your needs.

### Resize the image
This image uses the default size set by Singularity _(image size + 200M of padding)_ which is great for quick builds and pulls, but it is likely you'll need more space to accommodate your custom software stack.

In [3]:
ls -lsah | grep jupyter-base.img

669M -rwxr-xr-x.  1 vagrant vagrant 893M Sep 27 15:41 jupyter-base.img


When pulling from a Docker registry, you can use the `--size` flag to specify the built image size. Notice that Singularity isn't grabbing Docker layers from the registry, because the specified commit _(ae885c0a6226)_ has already been pulled. Singularity Docker cache is located in `$HOME/.singularity/docker`.

In [4]:
singularity pull --size 1500 --name "jupyter-ext.img" docker://jupyter/base-notebook

Initializing Singularity image subsystem
Opening image file: jupyter-ext.img
Creating 1500MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-ext.img
Docker image path: index.docker.io/jupyter/base-notebook:latest
Cache folder set to /home/vagrant/.singularity/docker
Importing: base Singularity environment
Importing: /home/vagrant/.singularity/docker/sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Importing: /home/vagrant/.singulari

In [5]:
singularity exec -e jupyter-ext.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec lab labextension labhub
migrate nbconvert nbextension notebook run serverextension troubleshoot trust


### Installing software _(the quick way)_
By default Singularity containers mounted as read-only volumes, which means you won't be able to add content or install software _(even as a privileged user)_, save for default or system-mounted paths. In order to add content you must run your Singularity command with the `--writable` flag.

For an interactive shell into your container, use the `shell` subcommand. The command below also passes the `-e` flag, which tells Singularity to strip the host environment before entering the container.

In [ ]:
sudo singularity shell -e --writable jupyter-ext.img

Singularity: Invoking an interactive shell within container...



Alternatively, you can use the `exec` subcommand to execute commands in your container without leaving your host environment.

In [10]:
sudo /usr/local/bin/singularity exec -e --writable jupyter-ext.img apt update
sudo /usr/local/bin/singularity exec -e --writable jupyter-ext.img apt -y install python3-dev python3-pip
sudo /usr/local/bin/singularity exec -e --writable jupyter-ext.img pip3 install --upgrade pip
sudo /usr/local/bin/singularity exec -e --writable jupyter-ext.img pip3 install seaborn

Hit:1 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu xenial InRelease
Hit:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
2 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binutils build-essential cpp cpp-5 dh-python dpkg-dev fakeroot file g++
  g++-5 gcc gcc-5 ifupdown iproute2 isc-dhcp-client isc-dhcp-common
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libalgorithm-merge-perl
  libasan2 libatm1 libatomic1 libc-dev-bin libc6-dev libcc1-0 libcilkrts5
  libdns-export162 libdpkg-perl libexpat1 libexpat1-dev libfakeroot
  libfile-fcntllock-perl libgcc-5-dev libgdbm3 libgmp10 libgomp1
  libisc-e

Get:47 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libubsan0 amd64 5.4.0-6ubuntu1~16.04.4 [95.3 kB]
Get:48 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libcilkrts5 amd64 5.4.0-6ubuntu1~16.04.4 [40.1 kB]
Get:49 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libmpx0 amd64 5.4.0-6ubuntu1~16.04.4 [9,766 B]
Get:50 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libquadmath0 amd64 5.4.0-6ubuntu1~16.04.4 [131 kB]
Get:51 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgcc-5-dev amd64 5.4.0-6ubuntu1~16.04.4 [2,237 kB]
Get:52 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 gcc-5 amd64 5.4.0-6ubuntu1~16.04.4 [8,577 kB]
Get:53 http://archive.ubuntu.com/ubuntu xenial/main amd64 gcc amd64 4:5.3.1-1ubuntu1 [5,244 B]
Get:54 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libstdc++-5-dev amd64 5.4.0-6ubuntu1~16.04.4 [1,426 kB]
Get:55 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 g++-5 amd64 5.4.0-6ubunt

Preparing to unpack .../libgdbm3_1.8.3-13.1_amd64.deb ...
Unpacking libgdbm3:amd64 (1.8.3-13.1) ...
7Progress: [ 13%] [########..................................................] 8Selecting previously unselected package perl-modules-5.22.
Preparing to unpack .../perl-modules-5.22_5.22.1-9_all.deb ...
Unpacking perl-modules-5.22 (5.22.1-9) ...
Selecting previously unselected package libperl5.22:amd64.
Preparing to unpack .../libperl5.22_5.22.1-9_amd64.deb ...
7Progress: [ 14%] [#########.................................................] 8Unpacking libperl5.22:amd64 (5.22.1-9) ...
Selecting previously unselected package perl.
Preparing to unpack .../perl_5.22.1-9_amd64.deb ...
Unpacking perl (5.22.1-9) ...
7Progress: [ 15%] [#########.................................................] 8Selecting previously unselected package libgmp10:amd64.
Preparing to unpack .../libgmp10_2%3a6.1.0+dfsg-2_amd64.deb ...
Unpacking libgmp10:amd64 (2:6.1.0+dfsg-2) ...
7Progress: [ 16%] [##########....

7Progress: [ 37%] [######################....................................] 8Selecting previously unselected package libmpx0:amd64.
Preparing to unpack .../libmpx0_5.4.0-6ubuntu1~16.04.4_amd64.deb ...
Unpacking libmpx0:amd64 (5.4.0-6ubuntu1~16.04.4) ...
7Progress: [ 38%] [#######################...................................] 8Selecting previously unselected package libquadmath0:amd64.
Preparing to unpack .../libquadmath0_5.4.0-6ubuntu1~16.04.4_amd64.deb ...
Unpacking libquadmath0:amd64 (5.4.0-6ubuntu1~16.04.4) ...
Selecting previously unselected package libgcc-5-dev:amd64.
Preparing to unpack .../libgcc-5-dev_5.4.0-6ubuntu1~16.04.4_amd64.deb ...
7Progress: [ 39%] [#######################...................................] 8Unpacking libgcc-5-dev:amd64 (5.4.0-6ubuntu1~16.04.4) ...
Selecting previously unselected package gcc-5.
Preparing to unpack .../gcc-5_5.4.0-6ubuntu1~16.04.4_amd64.deb ...
Unpacking gcc-5 (5.4.0-6ubuntu1~16.04.4) ...
7Progress: [ 40%] [##############

Preparing to unpack .../archives/rename_0.20-4_all.deb ...
Unpacking rename (0.20-4) ...
Processing triggers for libc-bin (2.23-0ubuntu9) ...
Processing triggers for systemd (229-4ubuntu19) ...
Setting up libatm1:amd64 (1:2.5.1-1.5) ...
7Progress: [ 62%] [####################################......................] 8Setting up libmnl0:amd64 (1.0.3-5) ...
Setting up mime-support (3.59ubuntu1) ...
7Progress: [ 63%] [#####################################.....................] 8Setting up libmpdec2:amd64 (2.4.2-1) ...
Setting up libsqlite3-0:amd64 (3.11.0-1ubuntu1) ...
7Progress: [ 64%] [######################################....................] 8Setting up libpython3.5-stdlib:amd64 (3.5.2-2ubuntu0~16.04.2) ...
Setting up python3.5 (3.5.2-2ubuntu0~16.04.2) ...
7Progress: [ 65%] [######################################....................] 8Setting up libpython3-stdlib:amd64 (3.5.1-3) ...
Setting up libgdbm3:amd64 (1.8.3-13.1) ...
7Progress: [ 66%] [#################################

Setting up python3-wheel (0.29.0-1) ...
7Progress: [ 99%] [##########################################################] 8Setting up dh-python (2.20151103ubuntu1.1) ...
Processing triggers for libc-bin (2.23-0ubuntu9) ...
Processing triggers for systemd (229-4ubuntu19) ...

78The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 184kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 16.9MB 107kB/s eta 0:00:01
    100% |████████████████████████████████| 47.9MB 27kB/s  eta 0:00:011
    100% |██████

Now seaborn is installed in your image.

In [13]:
singularity exec -e jupyter-ext.img pip3 freeze | grep seaborn

seaborn==0.8.1


### Installing Software _(the reproducible way)_
Shelling into your container and making ad-hoc changes is excellent for debugging and initial development, but it is considered bad practice as the steps needed to construct your software environment are not captured and cannot be reproduced.

To make durable, reproducible changes you need to build a spec file from which you can bootstrap your container. Bootstrapping must be done by a privileged user

In [15]:
cat jupyter-bootstrapped.def

BootStrap: docker
From: jupyter/base-notebook

%post
  echo "Installing dependencies..."
  apt update
  apt -y install python3-dev python3-pip
  pip3 install --upgrade pip
  echo "Installing seaborn..."
  pip3 install seaborn


In [19]:
singularity create --size 2000 jupyter-bootstrapped.img
sudo /usr/local/bin/singularity bootstrap jupyter-bootstrapped.img jupyter-bootstrapped.def

Initializing Singularity image subsystem
Opening image file: jupyter-bootstrapped.img
Creating 2000MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-bootstrapped.img
Sanitizing environment
Building from bootstrap definition recipe
Adding base Singularity environment to container
Docker image path: index.docker.io/jupyter/base-notebook:latest
Cache folder set to /root/.singularity/docker
Exploding layer: sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Exploding layer: sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Exploding layer: sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Exploding layer: sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Exploding layer: sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Exploding layer: sha256:b337aaee648d9f87e96fae8b24ae2dd887a2ded309b38dbee691fcdb040878cc.ta

Get:11 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3.5 amd64 3.5.2-2ubuntu0~16.04.2 [165 kB]
Get:12 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpython3-stdlib amd64 3.5.1-3 [6818 B]
Get:13 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 dh-python all 2.20151103ubuntu1.1 [74.1 kB]
Get:14 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3 amd64 3.5.1-3 [8710 B]
Get:15 http://archive.ubuntu.com/ubuntu xenial/main amd64 libgdbm3 amd64 1.8.3-13.1 [16.9 kB]
Get:16 http://archive.ubuntu.com/ubuntu xenial/main amd64 perl-modules-5.22 all 5.22.1-9 [2641 kB]
Get:17 http://archive.ubuntu.com/ubuntu xenial/main amd64 libperl5.22 amd64 5.22.1-9 [3371 kB]
Get:18 http://archive.ubuntu.com/ubuntu xenial/main amd64 perl amd64 5.22.1-9 [237 kB]
Get:19 http://archive.ubuntu.com/ubuntu xenial/main amd64 libgmp10 amd64 2:6.1.0+dfsg-2 [240 kB]
Get:20 http://archive.ubuntu.com/ubuntu xenial/main amd64 libmpfr4 amd64 3.1.4-1 [191 kB]
Get:21 http://archive.ubun

Preparing to unpack .../libpython3.5-minimal_3.5.2-2ubuntu0~16.04.2_amd64.deb ...
Unpacking libpython3.5-minimal:amd64 (3.5.2-2ubuntu0~16.04.2) ...
7Progress: [  2%] [##........................................................] 8Selecting previously unselected package libexpat1:amd64.
Preparing to unpack .../libexpat1_2.1.0-7ubuntu0.16.04.3_amd64.deb ...
Unpacking libexpat1:amd64 (2.1.0-7ubuntu0.16.04.3) ...
Selecting previously unselected package python3.5-minimal.
Preparing to unpack .../python3.5-minimal_3.5.2-2ubuntu0~16.04.2_amd64.deb ...
7Progress: [  3%] [##........................................................] 8Unpacking python3.5-minimal (3.5.2-2ubuntu0~16.04.2) ...
Selecting previously unselected package python3-minimal.
Preparing to unpack .../python3-minimal_3.5.1-3_amd64.deb ...
Unpacking python3-minimal (3.5.1-3) ...
7Progress: [  4%] [###.......................................................] 8Selecting previously unselected package mime-support.
Preparing to un

Preparing to unpack .../libc-dev-bin_2.23-0ubuntu9_amd64.deb ...
Unpacking libc-dev-bin (2.23-0ubuntu9) ...
7Progress: [ 27%] [################..........................................] 8Selecting previously unselected package linux-libc-dev:amd64.
Preparing to unpack .../linux-libc-dev_4.4.0-96.119_amd64.deb ...
Unpacking linux-libc-dev:amd64 (4.4.0-96.119) ...
Selecting previously unselected package libc6-dev:amd64.
Preparing to unpack .../libc6-dev_2.23-0ubuntu9_amd64.deb ...
7Progress: [ 28%] [#################.........................................] 8Unpacking libc6-dev:amd64 (2.23-0ubuntu9) ...
Selecting previously unselected package libisl15:amd64.
Preparing to unpack .../libisl15_0.16.1-1_amd64.deb ...
Unpacking libisl15:amd64 (0.16.1-1) ...
7Progress: [ 29%] [#################.........................................] 8Selecting previously unselected package cpp-5.
Preparing to unpack .../cpp-5_5.4.0-6ubuntu1~16.04.4_amd64.deb ...
Unpacking cpp-5 (5.4.0-6ubuntu1~16.04

Preparing to unpack .../libalgorithm-merge-perl_0.08-3_all.deb ...
Unpacking libalgorithm-merge-perl (0.08-3) ...
7Progress: [ 51%] [##############################............................] 8Selecting previously unselected package libexpat1-dev:amd64.
Preparing to unpack .../libexpat1-dev_2.1.0-7ubuntu0.16.04.3_amd64.deb ...
Unpacking libexpat1-dev:amd64 (2.1.0-7ubuntu0.16.04.3) ...
7Progress: [ 52%] [###############################...........................] 8Selecting previously unselected package libfile-fcntllock-perl.
Preparing to unpack .../libfile-fcntllock-perl_0.22-3_amd64.deb ...
Unpacking libfile-fcntllock-perl (0.22-3) ...
Selecting previously unselected package libpython3.5:amd64.
Preparing to unpack .../libpython3.5_3.5.2-2ubuntu0~16.04.2_amd64.deb ...
7Progress: [ 53%] [###############################...........................] 8Unpacking libpython3.5:amd64 (3.5.2-2ubuntu0~16.04.2) ...
Selecting previously unselected package libpython3.5-dev:amd64.
Preparing t

Setting up libmpx0:amd64 (5.4.0-6ubuntu1~16.04.4) ...
7Progress: [ 83%] [#################################################.........] 8Setting up libquadmath0:amd64 (5.4.0-6ubuntu1~16.04.4) ...
Setting up libgcc-5-dev:amd64 (5.4.0-6ubuntu1~16.04.4) ...
Setting up gcc-5 (5.4.0-6ubuntu1~16.04.4) ...
7Progress: [ 84%] [#################################################.........] 8Setting up gcc (4:5.3.1-1ubuntu1) ...
Setting up libstdc++-5-dev:amd64 (5.4.0-6ubuntu1~16.04.4) ...
7Progress: [ 85%] [##################################################........] 8Setting up g++-5 (5.4.0-6ubuntu1~16.04.4) ...
Setting up g++ (4:5.3.1-1ubuntu1) ...
7Progress: [ 86%] [##################################################........] 8update-alternatives: using /usr/bin/g++ to provide /usr/bin/c++ (c++) in auto mode
Setting up make (4.1-6) ...
Setting up libdpkg-perl (1.18.4ubuntu1.2) ...
7Progress: [ 87%] [###################################################.......] 8Setting up xz-utils (5.1.1alpha

In [20]:
singularity exec -e jupyter-bootstrapped.img pip3 freeze | grep seaborn

seaborn==0.8.1


# Docker, since shub bootstrap unsupported

In [ ]:
singularity pull --name "jupyter-base.img" docker://jupyter/base-notebook
singularity create --size 1500 jupyter-ext.img
singularity export jupyter-base.img | singularity import jupyter-ext.img

```
BootStrap: docker
From: jupyter/base-notebook

%post
  echo "Installing seaborn..."
  pip3 install seaborn
```

In [5]:
singularity help

USAGE: singularity [global options...] <command> [command options...] ...

GLOBAL OPTIONS:
    -d --debug    Print debugging information
    -h --help     Display usage summary
    -q --quiet    Only print errors
       --version  Show application version
    -v --verbose  Increase verbosity +1
    -x --sh-debug Print shell wrapper debugging information

GENERAL COMMANDS:
    help          Show additional help for a command
    selftest      Run some self tests to make sure Singularity is
                    installed and operating properly

CONTAINER USAGE COMMANDS:
    exec          Execute a command within container
    run           Launch a runscript within container
    shell         Run a Bourne shell within container
    test          Execute any test code defined within container

CONTAINER USAGE OPTIONS:
    see singularity help <command>

CONTAINER MANAGEMENT COMMANDS:
    bootstrap     Bootstrap a new Singularity image from scratch [root]
    copy          Copy files from y

In [10]:
rm jupyter-base.img

In [18]:
df -h

Filesystem                       Size  Used Avail Use% Mounted on
/dev/mapper/VolGroup00-LogVol00   38G   11G   28G  28% /
devtmpfs                         236M     0  236M   0% /dev
tmpfs                            245M     0  245M   0% /dev/shm
tmpfs                            245M  4.4M  241M   2% /run
tmpfs                            245M     0  245M   0% /sys/fs/cgroup
/dev/sda2                       1014M   65M  950M   7% /boot
tmpfs                             49M     0   49M   0% /run/user/1000
tmpfs                             16M     0   16M   0% /usr/local/var/singularity/mnt/session
